# World Bank International Debt Data

In this notebook, we are going to analyze international debt data collected by The World Bank. The dataset contains information about the amount of debt (in USD) owed by developing countries across several categories. We are going to find the answers to questions like: 

- **What is the total amount of debt that is owed by the countries listed in the dataset?**
- **Which country owns the maximum amount of debt and what does that amount look like?**
- **What is the average amount of debt owed by countries across different debt indicators?**

*The data used in this project is provided by The World Bank. It contains both national and regional debt statistics for several countries across the globe as recorded from 1970 to 2015.*

In [1]:
import os
from dotenv import load_dotenv

In [2]:
# Loading env file with enviroment variable
load_dotenv()
db_connection = os.getenv('url')

In [3]:
# Loading SQL extension for Jupyter Notebook
%load_ext sql

In [4]:
# Hiding connection from output
%config SqlMagic.displaycon=False

In [5]:
# Getting enviroment variable
%sql $db_connection

## 1. Exploring the dataset

First of all, we will describe our dataset and see what we will be analyzing

In [6]:
%%sql

SELECT
  table_name,
  column_name,
  data_type,
  character_maximum_length,
  numeric_precision,
  numeric_scale,
  column_default,
  is_nullable
FROM information_schema.columns
WHERE table_name = 'international_debt';

5 rows affected.


table_name,column_name,data_type,character_maximum_length,numeric_precision,numeric_scale,column_default,is_nullable
international_debt,debt,numeric,None,None,None,None,YES
international_debt,country_name,character varying,50,None,None,None,YES
international_debt,country_code,character varying,50,None,None,None,YES
international_debt,indicator_name,text,None,None,None,None,YES
international_debt,indicator_code,text,None,None,None,None,YES


We can see there are 5 columns in out dataset

1. Country Name
2. Country Code
3. Indicator Name
4. Indicator Code
5. Debt

Let's see the first ten rows

In [7]:
%%sql

SELECT *
FROM international_debt
LIMIT 10;

10 rows affected.


country_name,country_code,indicator_name,indicator_code,debt
Afghanistan,AFG,"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD,72894453.700000003
Afghanistan,AFG,"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD,53239440.100000001
Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,61739336.899999999
Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,49114729.399999999
Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,39903620.100000001
Afghanistan,AFG,"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD,39107845
Afghanistan,AFG,"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD,23779724.300000001
Afghanistan,AFG,"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD,13335820
Afghanistan,AFG,"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD,100847181.900000006
Afghanistan,AFG,"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD,72894453.700000003


## 2. Finding the number of distinct countries

We can only see data from Afghanistan, but now we want to know the number of different countries we have on the table. We will find that several countries appears many times, that is because a country can have many debts

In this section, we are going to extract **the number of unique countries present in the table**.

In [8]:
%%sql
SELECT COUNT(DISTINCT country_name) AS total_distinct_countries
FROM international_debt;

1 rows affected.


total_distinct_countries
124


## 3. Finding out the distinct debt indicators

As we saw in the first section, there is a column called <code>indicator_name</code> that briefly specifies the purpose of taking the debt. Just beside that column, there is another column called <code>indicator_code</code> which symbolizes the category of these debts. Knowing about these various debt indicators will help us to understand the areas in which a country can possibly be indebted to.

So far we know the following:

- There are **124** different countries in the dataset

In [9]:
%%sql

SELECT DISTINCT indicator_code AS distinct_debt_indicators
FROM international_debt
ORDER BY distinct_debt_indicators;

25 rows affected.


distinct_debt_indicators
DT.AMT.BLAT.CD
DT.AMT.DLXF.CD
DT.AMT.DPNG.CD
DT.AMT.MLAT.CD
DT.AMT.OFFT.CD
DT.AMT.PBND.CD
DT.AMT.PCBK.CD
DT.AMT.PROP.CD
DT.AMT.PRVT.CD
DT.DIS.BLAT.CD


## 4. Totaling the amount of debt owed by the countries

The financial debt of a particular country represents its economic state. But if we were to project this on an overall global scale, how will we approach it?

Let's switch gears from the debt indicators now and find out the total amount of debt (in USD) that is owed by the different countries. This will give us a sense of how the overall economy of the entire world is holding up.

We already know:

- There are **124** different countries in the dataset
- There are **25** different Debt Indicators

In [10]:
%%sql

SELECT ROUND(SUM(debt) / 1000000,2) AS total_debt
FROM international_debt;

1 rows affected.


total_debt
3079734.49


## 5. Country with the highest debt

Now that we have the exact total of the amounts of debt owed by several countries, let's now find out the country that owns the highest amount of debt along with the amount. 

Note that this debt is the sum of different debts owed by a country across several categories. This will help to understand more about the country in terms of its socio-economic scenarios. We can also find out the category in which the country owns its highest debt. But we will leave that for now.

We already know:

- There are **124** different countries in the dataset
- There are **25** different Debt Indicators
- The total amount of debt is **3,079,734.49 USD** (Notice that we divided the total sum of detb with 1000000 to use a smaller unit of measure)

In [11]:
%%sql

SELECT country_name, SUM(debt) AS total_debt
FROM international_debt
GROUP BY country_name
ORDER BY total_debt DESC
LIMIT 1;

1 rows affected.


country_name,total_debt
China,285793494734.200001568


## 6. Average amount of debt across indicators

So, it was *China*. 

We now have a brief overview of the dataset and a few of its summary statistics. We already have an idea of the different debt indicators in which the countries owe their debts. 

We can dig even further to find out on an average how much debt a country owes? This will give us a better sense of the distribution of the amount of debt across different indicators.

We already know:

- There are **124** different countries in the dataset
- There are **25** different Debt Indicators
- The total amount of debt is **3,079,734.49 USD** (Notice that we divided the total sum of detb with 1000000 to use a smaller unit of measure)
- The country with the highest debt is **China** with **285,793,494,734.20 USD**

In [12]:
%%sql

SELECT indicator_code AS debt_indicator, indicator_name, AVG(debt) AS average_debt
FROM international_debt
GROUP BY debt_indicator, indicator_name
ORDER BY average_debt DESC;

25 rows affected.


debt_indicator,indicator_name,average_debt
DT.AMT.DLXF.CD,"Principal repayments on external debt, long-term (AMT, current US$)",5904868401.499193612
DT.AMT.DPNG.CD,"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",5161194333.812658349
DT.DIS.DLXF.CD,"Disbursements on external debt, long-term (DIS, current US$)",2152041216.890243888
DT.DIS.OFFT.CD,"PPG, official creditors (DIS, current US$)",1958983452.859836046
DT.AMT.PRVT.CD,"PPG, private creditors (AMT, current US$)",1803694101.963265321
DT.INT.DLXF.CD,"Interest payments on external debt, long-term (INT, current US$)",1644024067.650806481
DT.DIS.BLAT.CD,"PPG, bilateral (DIS, current US$)",1223139290.398230108
DT.INT.DPNG.CD,"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",1220410844.421518983
DT.AMT.OFFT.CD,"PPG, official creditors (AMT, current US$)",1191187963.083064523
DT.AMT.PBND.CD,"PPG, bonds (AMT, current US$)",1082623947.653623188


## 7. The highest amount of principal repayments


We can see that the indicator <code>DT.AMT.DLXF.CD</code> tops the chart of average debt. This category includes repayment of long term debts. Countries take on long-term debt to acquire immediate capital.

An interesting observation in the above finding is that there is a huge difference in the amounts of the indicators after the second one. **This indicates that the first two indicators might be the most severe categories in which the countries owe their debts**.

We can investigate this a bit more so as to find out which country owes the highest amount of debt in the category of long term debts (<code>DT.AMT.DLXF.CD</code>). Since not all the countries suffer from the same kind of economic disturbances, this finding will allow us to understand that particular country's economic condition a bit more specifically.

We already know:

- There are **124** different countries in the dataset.
- There are **25** different Debt Indicators.
- The total amount of debt is **3,079,734.49 USD** (Notice that we divided the total sum of detb with 1000000 to use a smaller unit of measure).
- The country with the highest debt is **China** with **285,793,494,734.20 USD**.
- The highest average debt indicator is **Principal repayments on external debt, long-term (AMT, current US$)** with **5,904,868,401.50 USD**.
- The lowest average debt indicator is **PPG, other private creditors (INT, current US$)** with **34,250,651.24 USD**.

In [13]:
%%sql

SELECT country_name, indicator_name
FROM international_debt
WHERE debt = (SELECT MAX(debt)
             FROM international_debt
             WHERE indicator_code = 'DT.AMT.DLXF.CD');

1 rows affected.


country_name,indicator_name
China,"Principal repayments on external debt, long-term (AMT, current US$)"


## 8. The most common debt indicator

China has the highest amount of debt in the long-term debt (<code>DT.AMT.DLXF.CD</code>) category.

We saw that long-term debt is the topmost category when it comes to the average amount of debt. But is it the most common indicator in which the countries owe their debt? Let's find that out.

- There are **124** different countries in the dataset.
- There are **25** different Debt Indicators.
- The total amount of debt is **3,079,734.49 USD** (Notice that we divided the total sum of detb with 1000000 to use a smaller unit of measure).
- The country with the highest debt is **China** with **285,793,494,734.20 USD**.
- The highest average debt indicator is **Principal repayments on external debt, long-term (AMT, current US$)** with **5,904,868,401.50 USD**.
- The lowest average debt indicator is **PPG, other private creditors (INT, current US$)** with **34,250,651.24 USD**.
- China has the highest amount of debt in the long-term debt category

In [14]:
%%sql

SELECT indicator_code, COUNT(indicator_code) AS indicator_count
FROM international_debt
GROUP BY indicator_code
ORDER BY indicator_count DESC, indicator_code DESC;

25 rows affected.


indicator_code,indicator_count
DT.INT.OFFT.CD,124
DT.INT.MLAT.CD,124
DT.INT.DLXF.CD,124
DT.AMT.OFFT.CD,124
DT.AMT.MLAT.CD,124
DT.AMT.DLXF.CD,124
DT.DIS.DLXF.CD,123
DT.INT.BLAT.CD,122
DT.DIS.OFFT.CD,122
DT.AMT.BLAT.CD,122


## 9. Other viable debt issues and conclusion

There are a total of six debt indicators in which all the countries listed in our dataset have taken debt. The indicator <code>DT.AMT.DLXF.CD</code> is also there in the list. So, this gives us a clue that all these countries are suffering from a common economic issue. But that is not the end of the story, but just a part of the story.

Let's change tracks from <code>debt_indicators</code> now and focus on the amount of debt again. Let's find out the maximum amount of debt that each country has. With this, we will be in a position to identify the other plausible economic issues a country might be going through.

In this notebook, we took a look at debt owed by countries across the globe. We extracted a few summary statistics from the data and unraveled some interesting facts and figures. We also validated our findings to make sure the investigations are correct.

- There are **124** different countries in the dataset.
- There are **25** different Debt Indicators.
- The total amount of debt is **3,079,734.49 USD** (Notice that we divided the total sum of detb with 1000000 to use a smaller unit of measure).
- The country with the highest debt is **China** with **285,793,494,734.20 USD**.
- The highest average debt indicator is **Principal repayments on external debt, long-term (AMT, current US$)** with **5,904,868,401.50 USD**.
- The lowest average debt indicator is **PPG, other private creditors (INT, current US$)** with **34,250,651.24 USD**.
- China has the highest amount of debt in the long-term debt category.
- There are a total of six debt indicators in which all the countries listed in our dataset have taken debt.
    - DT.INT.OFFT.CD	124 countries
    - DT.INT.MLAT.CD	124 countries
    - DT.INT.DLXF.CD	124 countries
    - DT.AMT.OFFT.CD	124 countries
    - DT.AMT.MLAT.CD	124 countries
    - DT.AMT.DLXF.CD	124 countries

In [15]:
%%sql

SELECT country_name, MAX(debt) AS maximum_debt
FROM international_debt
GROUP BY country_name
ORDER BY maximum_debt DESC
LIMIT 10;

10 rows affected.


country_name,maximum_debt
China,96218620835.699996948
Brazil,90041840304.100006104
Russian Federation,66589761833.5
Turkey,51555031005.800003052
South Asia,48756295898.199996948
Least developed countries: UN classification,40160766261.599998474
IDA only,34531188113.199996948
India,31923507000.799999237
Indonesia,30916112653.799999237
Kazakhstan,27482093686.400001526


## 10. Conclusions

1. ### **What is the total amount of debt that is owed by the countries listed in the dataset?**

#### Top 3
- China 96,218,620,835.70 USD
- Brazil 90,041,840,304.10 USD
- Russian Federation 66,589,761,833.50


2. ### **Which country owns the maximum amount of debt and what does that amount look like?**
China with 285,793,494,734.20 USD

3. ### **What is the average amount of debt owed by countries across different debt indicators?**

#### Top 3
- DT.AMT.DLXF.CD	Principal repayments on external debt, long-term (AMT, current US$)	5,904,868,401.50 USD
- DT.AMT.DPNG.CD	Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)	5,161,194,333.81 USD
- DT.DIS.DLXF.CD	Disbursements on external debt, long-term (DIS, current US$)	2,152,041,216.90 USD
